In [1]:
from PIL import Image
from PIL import ImageDraw
import timeit
import glob
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from skimage.feature import greycomatrix, greycoprops
import matplotlib.image as mpimg 
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
import random
from PIL import ImageEnhance

In [2]:
map_8bit_to_3bit = [i // 32 for i in range(256)]
model = {}
model_test = {}
neighbor = 3
time_modelling_start = timeit.default_timer()
model["cap"] = []
model["tulis"] = []
model_test["cap"] = []
model_test["tulis"] = []
mean = {}
variance = {}
mean["tulis"] = []
variance["tulis"] = []
mean["cap"] = []
variance["cap"] = []
a1 =0
b1=0

In [3]:
def reset():
    global map_8bit_to_3bit
    global model
    global model_test
    global time_modelling_start
    global mean
    global variance
    global a1
    global b1
    map_8bit_to_3bit = [i // 32 for i in range(256)]
    model = {}
    model_test = {}
    neighbor = 3
    time_modelling_start = timeit.default_timer()
    model["cap"] = []
    model["tulis"] = []
    model_test["cap"] = []
    model_test["tulis"] = []
    mean = {}
    variance = {}
    mean["tulis"] = []
    variance["tulis"] = []
    mean["cap"] = []
    variance["cap"] = []
    a1 =0
    b1=0

In [4]:
#Pre Processing
def load_img(img_path):
    return Image.open(img_path).convert('L')

def get_img_size(img):
    return img.size

def print_img(img):
    plt.imshow(img)
    
def get_img_colors(img):
    return list(img.getdata())

def construct_img(img_colors, img_dimension, sampling_count):
    img_new = Image.new('RGB', (img_dimension), (255, 255, 255))
    draw = ImageDraw.Draw(img_new)
    
    img_new_width = img_dimension[0]
    img_new_height = img_dimension[1]

    sample_width = int(img_new_width/sampling_count)
    sample_height = int(img_new_height/sampling_count)
    loop_count = 0

    img_colors_len = len(img_colors)

    for w in range(0, img_new_width, sample_width):
        for h in range(0, img_new_height, sample_height):
            if(loop_count == img_colors_len):
                break
            
            current_color = (img_colors[loop_count], img_colors[loop_count], img_colors[loop_count])
            draw.rectangle((w, h, w+sample_width, h+sample_height), fill=current_color)
            loop_count += 1
    
    return img_new

def get_3bit_colors(img):
    img_colors = get_img_colors(img)
    loop_count = 0
    
    for img_color in img_colors:
        img_colors[loop_count] = map_8bit_to_3bit[img_color]
        
        loop_count += 1
        
    return img_colors

def get_img_matrix(img_colors):
    img_matrix = [] 
    loop_count = 0
    
    loop_count = 0
    img_square_dimension = int(math.sqrt(len(img_colors)))
    
    for row in range(img_square_dimension):
        temp_row = []
        for col in range(img_square_dimension):
            temp_row.append(img_colors[loop_count])
            
            loop_count += 1
        img_matrix.append(temp_row)
        
    return img_matrix

In [5]:
#Features Extraction
def feature_extract(img, sampling_count, class_name):
    glcm_component=[]
    img_3bit_colors = get_3bit_colors(img)
    img_matrix = get_img_matrix(img_3bit_colors)
    glcm_matrix=greycomatrix(img_matrix, distances=[1], angles=[0], levels=12, symmetric=False, normed=False)
    component = ['contrast', 'homogeneity', 'energy', 'correlation', 'ASM', 'dissimilarity']
    for x in component:
        glcm_component.append(greycoprops(glcm_matrix, x)[0][0])
    glcm_component.append(class_name)
    return(glcm_component)

In [6]:
#Modelling
def get_class_names(training_folder_path):
    return os.listdir(training_folder_path)

def make_a_model(class_name, features, learning_rate):
    random_splitter = random.uniform(0, 1)
    if(random_splitter <= learning_rate):
        model[class_name].append(features)
    else:
        model_test[class_name].append(features)
        
def to_dataframe(model):
    models = pd.DataFrame(model['cap'] + model['tulis'], columns=['contrast', 'homogeneity', 'energy', 'correlation', 'ASM', 'dissimilarity', 'class'])
    return models

In [7]:
#Classification
def naive_bayes(image, cap_probability, tulis_probability):
    prob = 1
    probability = 0.00
    for class_name in class_names:
        if(class_name=="tulis"):
            prob=tulis_probability
        elif(class_name=="cap"):
            prob=cap_probability
        for i in range(5):
            a = (image[i]-mean[class_name][i])**2/(2*variance[class_name][i])
            b = 1/(np.sqrt(2*3.14*variance[class_name][i]))
            prob=prob * (b* np.exp(0-a))
       
        if(prob>probability):
            probability = prob
            probability_class = class_name
        prob=1
    return probability_class

In [8]:
#Testing

def testing(cap_probability, tulis_probability):
    right = 0
    total = 0
    for class_name in class_names:
        for image in model_test[class_name]:
            predict = naive_bayes(image, cap_probability, tulis_probability)
            class_image = class_name
            if(predict == class_name):
                   right+=1
            total +=1
    accuracy = (right/total) * 100
    print("Accuracy: " + str(accuracy) +"%\n")
    
    
def testing_out(sampling_count):
    testing_img_paths = glob.glob(test_folder_path + '/' + img_type)
    for testing_img_path in testing_img_paths:
        imgs = load_img(testing_img_path)
        predict_class = knn(imgs, sampling_count, neighbor)
        print("Categorized as " + predict_class)
        plt.imshow(Image.open(testing_img_path))    

In [9]:
def testings():
    global a1, b1
    total={}
    total["tulis"] = models.loc[models['class'] == 'tulis']
    total["cap"] = models.loc[models['class'] == 'cap']
    mean["tulis"] = []
    variance["tulis"] = []
    mean["cap"] = []
    variance["cap"] = []
    tulis_probability = (total["tulis"].count()/models.count())[0]
    cap_probability = (total["cap"].count()/models.count())[0]
    a1=tulis_probability
    b1=cap_probability
    for a in ["tulis", "cap"]:
        for b in ['contrast', 'homogeneity', 'energy', 'correlation', 'ASM']:
            mean[a].append(total[a][b].mean())
            variance[a].append(total[a][b].var())
    
    testing(cap_probability, tulis_probability)

In [10]:
#Validation
def validate(sampling_count, a, b):
   
    right = 0
    total = 0
    for class_name in class_names:
        validate_img_paths = glob.glob(validation_folder_path + class_name + '/' +img_type)
        for images in validate_img_paths:
            img = load_img(images)
            enhancer = ImageEnhance.Sharpness(img)
            img = enhancer.enhance(1.5)
            training_img_class_name = class_name
            image = feature_extract(img, sampling_count, training_img_class_name)
            predict = naive_bayes(image,a,b)
            class_image = class_name
            if(predict == class_name):
                   right+=1
            total +=1
        accuracy = (right/total) * 100
    print("Accuracy: " + str(accuracy) +"%\n")

In [15]:
#Main

learning_rate = 0.8
sampling_count = 128
img_type = '*.jpg'
root_path = './'
training_folder_path = root_path + 'glcm/training/'
validation_folder_path = root_path + 'glcm/validation/'
test_folder_path = root_path + 'glcm/test/'

class_names = get_class_names(training_folder_path)

for j in range(3):
    for i in range(10):
        for class_name in class_names:
            training_img_paths = glob.glob(training_folder_path + class_name + '/' + img_type)
            for training_img_path in training_img_paths:
                training_img = load_img(training_img_path)
                enhancer = ImageEnhance.Sharpness(training_img)
                training_img = enhancer.enhance(1.5)
                training_img_class_name = class_name
                feature = feature_extract(training_img, sampling_count, training_img_class_name)
                make_a_model(class_name, feature, learning_rate)

        models = to_dataframe(model)
        models_test = to_dataframe(model_test)
        print("Epoch ke-"+ str(i+1) + " ")
        testings()
        
    print("Iterasi ke-"+ str(j+1) + " ")
    validate(128,a1,b1)
    time_modelling_stop = timeit.default_timer()
    print('Time elapsed to make this model is ' + str(time_modelling_stop - time_modelling_start) + ' seconds.')
    reset()

Epoch ke-1 
Accuracy: 78.21782178217822%

Epoch ke-2 
Accuracy: 74.17840375586854%

Epoch ke-3 
Accuracy: 74.61300309597523%

Epoch ke-4 
Accuracy: 73.1934731934732%

Epoch ke-5 
Accuracy: 74.08088235294117%

Epoch ke-6 
Accuracy: 74.24242424242425%

Epoch ke-7 
Accuracy: 75.51813471502591%

Epoch ke-8 
Accuracy: 74.97155858930603%

Epoch ke-9 
Accuracy: 75.65656565656566%

Epoch ke-10 
Accuracy: 74.93138151875571%

Iterasi ke-1 
Accuracy: 76.953125%

Time elapsed to make this model is 554.6241404 seconds.
Epoch ke-1 
Accuracy: 71.875%

Epoch ke-2 
Accuracy: 73.68421052631578%

Epoch ke-3 
Accuracy: 71.29032258064515%

Epoch ke-4 
Accuracy: 72.41379310344827%

Epoch ke-5 
Accuracy: 72.86821705426357%

Epoch ke-6 
Accuracy: 71.49758454106279%

Epoch ke-7 
Accuracy: 72.29080932784636%

Epoch ke-8 
Accuracy: 72.55594817432274%

Epoch ke-9 
Accuracy: 71.07180020811654%

Epoch ke-10 
Accuracy: 72.55813953488372%

Iterasi ke-2 
Accuracy: 75.390625%

Time elapsed to make this model is 507.640

In [12]:
a1

0